## **챗 모델과 프롬프트 템플릿을 사용해 간단한 LLM 애플리케이션 구축하기**

- [**언어 모델(Language Models)**](https://docs.langchain.com/docs/concepts/chat_models) 사용법  
- [**프롬프트 템플릿(Prompt Templates)**](https://docs.langchain.com/docs/concepts/prompt_templates) 사용법  
- [**LangSmith**](https://docs.smith.langchain.com)를 사용한 애플리케이션 디버깅 및 추적  

**LangSmith 설정**

LangChain을 사용한 애플리케이션은 **여러 단계의 LLM 호출**을 포함할 수 있습니다.  
애플리케이션이 **복잡해질수록** 내부에서 어떤 일이 일어나는지 **추적(Trace)**하는 것이 중요합니다.

가장 좋은 방법은 **[LangSmith](https://smith.langchain.com)**를 사용하는 것입니다.

1. **LangSmith 가입 및 설정**  
   - 위 링크에서 가입하세요.
<p></p>

2. **환경 변수 설정**
```
os.environ["LANGCHAIN_TRACING_V2"] = "true"
```

In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv

In [ ]:
# LangChain 추적(Tracing) 설정 활성화

## 언어 모델 사용하기

LangChain은 다양한 언어 모델을 지원하며, 이들을 서로 교체하여 사용할 수 있습니다.

In [ ]:
# GPT-4o-mini 모델을 사용하여 ChatOpenAI 인스턴스를 생성

ChatModels은 LangChain Runnables의 인스턴스로, 표준화된 인터페이스를 통해 상호작용할 수 있습니다. 모델을 간단히 호출하려면 `.invoke` 메서드에 Messages 목록을 전달하면 됩니다.

In [ ]:
# 메시지 목록을 생성
    # 시스템 메시지: 모델에게 수행할 작업이나 역할을 지시합니다.
    # 사용자 메시지: 사용자가 모델에 보낼 실제 입력 내용입니다.

LangSmith를 활성화했다면 이 실행이 LangSmith에 로깅되었으며, [LangSmith 추적](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r)을 확인할 수 있습니다. LangSmith 추적은 [토큰](/docs/concepts/tokens/) 사용 정보, 지연 시간(latency), [표준 모델 매개변수](/docs/concepts/chat_models/#standard-parameters)(예: temperature) 및 기타 정보를 제공합니다.


ChatModel은 입력으로 [메시지](/docs/concepts/messages/) 객체를 받고 출력으로 메시지 객체를 생성합니다. 메시지 객체는 텍스트 내용 외에도 대화의 [역할](/docs/concepts/messages/#role)을 전달하며, [도구 호출](/docs/concepts/tool_calling/) 및 토큰 사용량과 같은 중요한 데이터를 포함합니다.

LangChain은 문자열이나 [OpenAI 형식](/docs/concepts/messages/#openai-format)을 통해 채팅 모델 입력을 지원합니다. 다음 예제는 모두 동일한 기능을 수행합니다:

```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

### 스트리밍

채팅 모델은 [Runnables](/docs/concepts/runnables/)이기 때문에 비동기 및 스트리밍 호출 모드를 포함한 표준 인터페이스를 제공합니다. 이를 통해 채팅 모델로부터 개별 토큰을 스트리밍할 수 있습니다.

## 프롬프트 템플릿 (Prompt Templates)

우리는 메시지 목록을 언어 모델에 직접 전달합니다.  일반적으로 이 목록은 사용자 입력과 애플리케이션 로직의 조합으로 구성됩니다. 애플리케이션 로직은 사용자 입력(raw user input)을 받아 언어 모델에 전달할 메시지 목록으로 변환합니다. 일반적인 변환 과정에는 시스템 메시지를 추가하거나 사용자 입력을 템플릿에 맞게 포맷하는 작업이 포함됩니다.  

[프롬프트 템플릿](/docs/concepts/prompt_templates/)은 LangChain에서 이러한 변환을 돕기 위해 설계된 개념입니다. 프롬프트 템플릿은 사용자 입력(raw user input)을 받아 언어 모델에 전달할 준비가 된 데이터(프롬프트)로 반환합니다.

프롬프트 템플릿은 두 개의 사용자 변수를 입력으로 받습니다:

- **`language`**: 번역할 대상 언어  
- **`text`**: 번역할 텍스트  

`ChatPromptTemplate`은 하나의 템플릿에서 여러 메시지 역할을 지원합니다. `language` 매개변수는 시스템 메시지에 포맷되며, 사용자의 `text`는 사용자 메시지에 포맷됩니다.  
이 프롬프트 템플릿의 입력은 dictionary 입니다.

두 개의 메시지로 구성된 `ChatPromptValue`를 반환하는 것을 볼 수 있습니다. 메시지에 직접 액세스하려면 다음을 수행합니다.

마지막으로, 포맷된 프롬프트에서 채팅 모델을 호출할 수 있습니다.


프롬프트 템플릿은 사용자 입력과 매개변수를 언어 모델에 대한 지침으로 변환하는 데 도움을 줍니다. 이를 통해 모델이 컨텍스트를 이해하고 관련성 있고 일관된 언어 기반 출력을 생성하도록 유도할 수 있습니다.  

프롬프트 템플릿은 **딕셔너리** 형태의 입력을 받습니다. 여기서 각 키(key)는 프롬프트 템플릿에서 채워야 할 변수를 나타냅니다.  

프롬프트 템플릿은 **PromptValue**를 출력합니다. 이 **PromptValue**는 LLM 또는 ChatModel에 전달될 수 있으며, 문자열(string) 또는 메시지 목록(list of messages)으로 변환(cast)될 수도 있습니다. **PromptValue**가 존재하는 이유는 문자열과 메시지 형식 간 전환을 쉽게 하기 위함입니다.  

### 프롬프트 템플릿의 유형  

1. **String PromptTemplates (문자열 프롬프트 템플릿)**  
   - 이 프롬프트 템플릿은 단일 문자열을 포맷하는 데 사용되며, 일반적으로 더 간단한 입력에 사용됩니다.  
   - 예를 들어, 프롬프트 템플릿을 구성하고 사용하는 일반적인 방법은 다음과 같습니다:  

In [ ]:
# chain 작성 & invoke

---------------
동일한 결과를 얻는 다른 방법

In [ ]:
# from_template 이용
# prompt invoke + model invoke

2. **채팅 프롬프트 템플릿(ChatPromptTemplates)**
    - 이 프롬프트 템플릿은 **메시지 목록을 포맷**하는 데 사용됩니다. 이러한 "템플릿"은 자체적으로 여러 개의 템플릿 목록으로 구성됩니다.  
    - 예를 들어, **ChatPromptTemplate**를 구성하고 사용하는 일반적인 방법은 다음과 같습니다:

위의 예에서 ChatPromptTemplate은 호출 시 두 개의 메시지를 구성합니다. 첫 번째는 시스템 메시지로, 포맷할 변수가 없습니다. 두 번째는 HumanMessage로, 사용자가 전달하는 topic 변수로 포맷됩니다.

## 메시지 플레이스홀더(MessagesPlaceholder)  

이 프롬프트 템플릿은 특정 위치에 **메시지 목록을 추가**하는 역할을 합니다.  

위의 **ChatPromptTemplate** 예제에서 두 개의 메시지를 문자열로 각각 포맷하는 방법을 확인했습니다. 하지만 특정 위치에 사용자로부터 받은 **메시지 목록**을 삽입하고 싶다면 어떻게 해야 할까요?  

이럴 때 **MessagesPlaceholder**를 사용합니다.

이렇게 하면 두 개의 메시지 목록이 생성됩니다. 첫 번째는 **시스템 메시지**이고, 두 번째는 우리가 전달한 **HumanMessage**입니다. 만약 5개의 메시지를 전달했다면, 총 6개의 메시지가 생성됩니다 (시스템 메시지 1개 + 전달된 5개 메시지).  

이 방법은 특정 위치에 **메시지 목록을 삽입**할 때 유용합니다.  

**MessagesPlaceholder** 클래스를 명시적으로 사용하지 않고도 동일한 작업을 수행할 수 있는 다른 방법은 다음과 같습니다:

## **스트리밍 (Streaming)**  

**채팅 모델(Chat Models)** 은 **Runnables**이기 때문에 비동기(async) 및 **스트리밍(streaming)** 호출 모드를 포함한 표준 인터페이스를 제공합니다.  

이를 통해 채팅 모델로부터 **개별 토큰(token)** 을 스트리밍 방식으로 받을 수 있습니다:

**채팅 모델 응답을 스트리밍하는 방법**  

모든 채팅 모델(Chat Models)은 **Runnable 인터페이스**를 구현합니다. 이 인터페이스는 표준 실행 메서드(예: `ainvoke`, `batch`, `abatch`, `stream`, `astream`, `astream_events`)의 기본 구현을 제공합니다.  

기본 스트리밍 구현은 **Iterator**(또는 비동기 스트리밍의 경우 **AsyncIterator**)를 제공하며, 이는 **기본 채팅 모델 공급자**로부터 최종 출력을 단일 값으로 반환합니다.

#### **동기 스트리밍 (Sync Streaming)**  

아래에서는 `|` 기호를 사용하여 토큰(token) 간의 구분자를 시각화합니다.

#### **비동기 스트리밍(Async Streaming)**

#### **동기/비동기 차이점 요약**

| **특징**       | **Sync Streaming**           | **Async Streaming**        |
|---------------|-------------------------------|----------------------------|
| **실행 방식**  | 동기 (Blocking)              | 비동기 (Non-blocking)      |
| **병렬 처리**  | 불가능                       | 가능                       |
| **복잡성**    | 낮음                          | 높음                       |
| **사용 사례**  | 소규모, 단순 애플리케이션     | 대규모, 고성능 애플리케이션 |
| **예시 키워드**| `stream()`                   | `astream()`                |
